In [1]:
import sys
sys.path.append('../../..')

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd, numpy as np, gc
from tqdm import tqdm
import joblib
import xgboost as xgb
import pickle
pd.set_option('display.max_columns', 500)


from utils.util import *
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
from utils.dataiter import Dataiter
from utils.preprocessing import *
from utils.target_encode import MTE_one_shot

import core.config as conf

In [2]:
path = f'{conf.raw_lzo_path}/part-00000'
train = read_data(path)
path = f'{conf.raw_lzo_path}/part-00001'
train2 = read_data(path)
path = f'{conf.raw_lzo_path}/part-00002'
valid = read_data(path)
gc.collect()
save_memory( train )

In [6]:
TARGET = 'like'
# ypred['like'] = valid[[TARGET+'_timestamp']].applymap(lambda x : 1 if x > 0 else 0)

In [7]:
import numpy as np
class MTE_one_shot:
    
    def __init__(self, folds, smooth, seed=42):
        self.folds = folds
        self.seed = seed
        self.smooth = smooth
        # self.agg_all = pd.DataFrame()
        
    def fit_transform(self, train, x_col, y_col, y_mean=None, out_col = None, out_dtype=None):
        
        self.y_col = y_col
        np.random.seed(self.seed)
        
        if 'fold' not in train.columns:
            fsize = len(train)//self.folds
            train['fold'] = 1
            train['fold'] = train['fold'].cumsum()
            train['fold'] = train['fold']//fsize
            train['fold'] = train['fold']%self.folds
        
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'TE_{tag}_{self.y_col}'
        
        if y_mean is None:
            y_mean = train[y_col].mean()#.compute().astype('float32')
        self.mean = y_mean # mean도 누적해서 바꿔주면 좋을듯
        
        cols = ['fold',x_col] if isinstance(x_col,str) else ['fold']+x_col
        
        agg_each_fold = train.groupby(cols).agg({y_col:['count','sum']}).reset_index()
        agg_each_fold.columns = cols + ['count_y','sum_y']
        
        agg_all = agg_each_fold.groupby(x_col).agg({'count_y':'sum','sum_y':'sum'}).reset_index()
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all.columns = cols + ['count_y_all','sum_y_all']
        
        agg_each_fold = agg_each_fold.merge(agg_all,on=x_col,how='left')
        agg_each_fold['count_y_all'] = agg_each_fold['count_y_all'] - agg_each_fold['count_y']
        agg_each_fold['sum_y_all'] = agg_each_fold['sum_y_all'] - agg_each_fold['sum_y']
        agg_each_fold[out_col] = (agg_each_fold['sum_y_all']+self.smooth*self.mean)/(agg_each_fold['count_y_all']+self.smooth)
        agg_each_fold = agg_each_fold.drop(['count_y_all','count_y','sum_y_all','sum_y'],axis=1)
        
        agg_all[out_col] = (agg_all['sum_y_all']+self.smooth*self.mean)/(agg_all['count_y_all']+self.smooth)
        agg_all = agg_all.drop(['count_y_all','sum_y_all'],axis=1)
        
        if hasattr(self, 'agg_all'):
            print('train2')
            self.agg_all = pd.concat([self.agg_all, agg_all])
            
        else:
            print('train1')
            self.agg_all = agg_all
        
        self.agg_all = self.agg_all.drop_duplicates(x_col, keep='last')
        # agg_all.to_csv('agg_all.csv', index=False)

        train.columns
        cols = ['fold',x_col] if isinstance(x_col,str) else ['fold']+x_col
        train = train.merge(agg_each_fold,on=cols,how='left')
        del agg_each_fold
        #self.agg_each_fold = agg_each_fold
        #train[out_col] = train.map_partitions(lambda cudf_df: cudf_df[out_col].nans_to_nulls())
        train[out_col] = train[out_col].fillna(self.mean)
        
        if out_dtype is not None:
            train[out_col] = train[out_col].astype(out_dtype)
        return train
    
    def transform(self, test, x_col, out_col = None, out_dtype=None):
        
        # self.agg_all = pd.read_csv('agg_all.csv')
        print(len(self.agg_all))
        
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'TE_{tag}_{self.y_col}'
        test = test.merge(self.agg_all,on=x_col,how='left')
        test[out_col] = test[out_col].fillna(self.mean)
        # test[out_col] = test[out_col].fillna(0)
        if out_dtype is not None:
            test[out_col] = test[out_col].astype(out_dtype)
        return test

In [12]:
def set_dataframe_types(df, train):
    df['id']   = np.arange( df.shape[0] )
    df['id']   = df['id'].astype(np.uint32)

    if train:
        df['reply_timestamp']   = df['reply_timestamp'].fillna(0)
        df['retweet_timestamp'] = df['retweet_timestamp'].fillna(0)
        df['comment_timestamp'] = df['comment_timestamp'].fillna(0)
        df['like_timestamp']    = df['like_timestamp'].fillna(0)

        df['reply_timestamp']   = df['reply_timestamp'].astype(np.uint32)
        df['retweet_timestamp'] = df['retweet_timestamp'].astype(np.uint32)
        df['comment_timestamp'] = df['comment_timestamp'].astype(np.uint32)
        df['like_timestamp']    = df['like_timestamp'].astype(np.uint32)

    df['tweet_timestamp']         = df['tweet_timestamp'].astype( np.uint32 )
    df['creator_follower_count']  = df['creator_follower_count'].astype( np.uint32 )
    df['creator_following_count'] = df['creator_following_count'].astype( np.uint32 )
    df['creator_account_creation']= df['creator_account_creation'].astype( np.uint32 )
    df['engager_follower_count']  = df['engager_follower_count'].astype( np.uint32 )
    df['engager_following_count'] = df['engager_following_count'].astype( np.uint32 )
    df['engager_account_creation']= df['engager_account_creation'].astype( np.uint32 )

    return df



def preprocess(df, target, train):
    df = set_dataframe_types(df, train)
    # df = df.set_index('id')
    # df.columns = conf.raw_features + conf.labels
    df = df.drop('text_tokens', axis=1)
    
    df = feature_extraction(df, features=conf.used_features, train=train) # extract 'used_features'
    cols = []
    print('target_encode')
    for c in tqdm([
        # ['engager_id'],
        ['engager_id','tweet_type','language'],
        # ['creator_id'],
        # ['domains','media','tweet_type','language']
        ]):
        out_col = 'TE_'+'_'.join(c)+'_'+target
        encoder_path = f'{out_col}.pkl'
        if os.path.exists(encoder_path):
            with open(encoder_path, 'rb') as f:
                encoder = pickle.load(f)
        else:
            encoder = MTE_one_shot(folds=5,smooth=20)

        if train:
            
            df = encoder.fit_transform(df, c, target, out_col=out_col, out_dtype='float32')
            with open(encoder_path, 'wb') as f:
                pickle.dump(encoder, f)
        else:
                df = encoder.transform(df, c, out_col=out_col, out_dtype='float32')
        

        cols.append(out_col)
        del encoder


    return df

In [13]:

train = preprocess(train, TARGET, True)


  0%|          | 0/1 [00:00<?, ?it/s]target_encode
train1
100%|██████████| 1/1 [00:16<00:00, 16.12s/it]


In [15]:
train2 = preprocess(train2, TARGET, True)


  0%|          | 0/1 [00:00<?, ?it/s]target_encode
train2
100%|██████████| 1/1 [00:19<00:00, 19.08s/it]


In [16]:
valid = preprocess(valid, TARGET, False)


  0%|          | 0/1 [00:00<?, ?it/s]target_encode
4740024
100%|██████████| 1/1 [00:04<00:00,  4.65s/it]


In [17]:
label_names = ['reply', 'retweet', 'comment', 'like']
DONT_USE = ['timestamp','creator_account_creation','engager_account_creation','engage_time',
            'creator_account_creation', 'engager_account_creation',
            'fold','tweet_id', 
            'tr','dt_day','','ypred',
            'engager_id','creator_id','engager_is_verified',
            'elapsed_time',
            'links','domains','hashtags0','hashtags1',
            'hashtags','tweet_hash','dt_second','id',
            'tw_hash0',
            'tw_hash1',
            'tw_rt_uhash',
            'same_language', 'nan_language','language',
            'tw_hash', 'tw_freq_hash','tw_first_word', 'tw_second_word', 'tw_last_word', 'tw_llast_word',
            'ypred','creator_count_combined','creator_user_fer_count_delta_time','creator_user_fing_count_delta_time','creator_user_fering_count_delta_time','creator_user_fing_count_mode','creator_user_fer_count_mode','creator_user_fering_count_mode'
            'reply', 'retweet', 'comment', 'like', 'pred'
            
           ]
DONT_USE += label_names
DONT_USE += conf.labels


In [18]:
xgb_parms = { 
                'max_depth':8, 
                'learning_rate':0.025, 
                'subsample':0.85,
                'colsample_bytree':0.35, 
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'tree_method':'gpu_hist',
                #'predictor': 'gpu_predictor',
                'seed': 1,
            }
LR = [0.05,0.03,0.07,0.01]
xgb_parms['learning_rate'] = LR[3]
RMV = [c for c in DONT_USE if c in train.columns]
dtrain = xgb.DMatrix(data=train.drop(RMV, axis=1) ,label=train[TARGET].values)
model = xgb.train(xgb_parms, 
                        dtrain=dtrain,
                        num_boost_round=500,
                        ) 

In [19]:
train.drop(RMV, axis=1).columns

Index(['tweet_type', 'creator_follower_count', 'creator_following_count',
       'media', 'tweet_timestamp', 'dt_dow', 'dt_hour', 'len_domains',
       'TE_engager_id_tweet_type_language_like'],
      dtype='object')

In [24]:
RMV = [c for c in DONT_USE if c in valid.columns]
dvalid = xgb.DMatrix(data=valid.drop(RMV, axis=1) )
valid['pred'] = model.predict(dvalid)

In [25]:
valid.columns

Index(['creator_id', 'engager_id', 'tweet_id', 'tweet_type', 'language',
       'creator_follower_count', 'creator_following_count', 'domains', 'media',
       'tweet_timestamp', 'dt_day', 'dt_dow', 'dt_hour', 'len_domains',
       'TE_engager_id_tweet_type_language_like', 'pred'],
      dtype='object')

In [30]:
valid

,creator_id,engager_id,tweet_id,tweet_type,language,creator_follower_count,creator_following_count,domains,media,tweet_timestamp,dt_day,dt_dow,dt_hour,len_domains,TE_engager_id_tweet_type_language_like,pred
0,18F85439674B688B152D2BA381F6CD2B,B912AC07692898D015B1FBCF3FF08080,C6EA8E85E46040F39438B2E21BCEBA48,2,46,1139,634,0,5,1613046306,11,3,12,0,0.426697,0.615871
1,F9E2F9F8D850C98B63DC03ADF94372F7,F9D6ADEACCFB63EB768F8A6443EB96F7,03D901586D1F6C79EA722026DCCF2491,2,19,11350,195,0,5,1613133850,12,4,12,0,0.398207,0.498268
2,8609DF70010D0A970086BCB52439E28D,BFB6BBBCBCCF70AB06BA8B06CB4F4B03,1A66CE8117EA397B635ED5B476B4712F,0,48,2083,538,0,0,1613221565,13,5,13,0,0.398207,0.363982
3,C550F2FE483F4EBB60CEF1D0CE9BF4FF,725752EB0066B1A563A5BEB3F1688BFE,2C16025ED6BF3893D656161E94D044ED,2,61,2231,2980,0,0,1612804536,8,0,17,0,0.398207,0.429048
4,CD19DCDDF4F32347D88C9F439A87FB0E,3D4AC79A1ADF2DF38C50929265D19CAA,D8CDCC2D06BEFC1FED6E6A5BA0F87A5F,2,46,19732,9500,0,5,1613047958,11,3,12,0,0.398207,0.470106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032652,0543370735E7090F7B2B3071C6A2E91D,D23C8834E8856155EA2513DC3FBB2D5A,BF71A491B5539752927DC8C8ACE8C2D1,2,19,11710,166,0,0,1614091454,23,1,14,0,0.379244,0.423858
3032653,35BC2642DA0FE690E66EB8801E67823A,128D0813732451D6D0B09EBBE3E34AFF,A42D80B94CC64BD9996BD45A939A276F,2,19,57866,48415,0,5,1613103596,12,4,4,0,0.398207,0.409211
3032654,B0CFCE419E235CE73DD1E1BAA9A661B2,BBB6E4448B96147F5D213C557C17E005,EDF8B1C5C00A8A3166E9AF1B483EC6A0,2,19,129632,360,519,9,1613417226,15,0,19,1,0.398207,0.272296
3032655,CBF323E1A477EA353DB78916F6AB00A8,0CF299C831ECAC9A21A9AF2B140E5B6A,B0DCB3AF2DBAFD78A22D113B1EEAB4DE,0,19,93,90,0,0,1612996196,10,2,22,0,0.398207,0.439516


In [31]:
valid['pred']

0          0.615871
1          0.498268
2          0.363982
3          0.429048
4          0.470106
             ...   
3032652    0.423858
3032653    0.409211
3032654    0.272296
3032655    0.439516
3032656    0.296145
Name: pred, Length: 3032657, dtype: float32

In [40]:
rce_like = compute_rce(valid['pred'], ypred['like'])
rce_like

7.965242661748306

In [39]:
ap_like = average_precision_score(ypred['like'], valid['pred'])
ap_like

0.5835477414410601